In [1]:
from sqlalchemy.engine import URL
from sqlalchemy import Column, Integer, String, DateTime, Text, ForeignKey, Uuid, SmallInteger, DECIMAL
from sqlalchemy.orm import declarative_base
from datetime import datetime
from sqlalchemy.orm import sessionmaker
from home_security.data.postgresql.database import get_session, create_tables, seed_db
from home_security.data.postgresql.models import Household, Camera, ImageSet, IntrusionImage
from home_security.data.storage import StorageDriver
from PIL import Image
from typing import List
from home_security.data.minio.minio_client import Minio
import uuid
from home_security.data.postgresql.database import Base, engine
Base.metadata.drop_all(engine) # this deletes all tables !!!
minio = Minio(endpoint="localhost:9000", access_key='homesecurity', secret_key='homesecuritybyjohn')
session = get_session()
create_tables()
seed_db()

storage_driver = StorageDriver(minio=minio, session=session)

In [2]:
filename = "D01-1_20240605104950.jpg"
image = Image.open(f'../images/new_images/denning/{filename}')

In [5]:
storage_driver.add_image_set(images=[image], camera_id='D01', datetime_str='20250223144702')

In [6]:
storage_driver.get_uninferenced_image_sets()

[1, 2]

In [11]:
from sqlalchemy import select
query = select(IntrusionImage.id, ImageSet.received_date, Household.name, Household.id, Camera.caption, Camera.id).join(IntrusionImage).join(ImageSet).join(Household).join(Camera)
session.execute(query).all()

InvalidRequestError: Don't know how to join to <Mapper at 0x215a33b6cb0; Household>. Please use the .select_from() method to establish an explicit left side, as well as providing an explicit ON clause if not present already to help resolve the ambiguity.